In [14]:
import pandas as pd
import numpy as np
import sys
import time

sys.path.append('../../../')
from utils import modelling as mod
from utils import utils
from utils import paths
import wandb

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold, cross_val_score, cross_validate

In [3]:
df = pd.read_csv('/Users/tudor/Documents/phd/coding/project_CF/data/final_df_aberdeen.csv')
# df = pd.read_csv('/gws/nopw/j04/ai4er/users/ts809/era5_final/final_df_aberdeen.csv')    # JASMIN

weights = mod.CalcClassWeights(df['floods'])
df_to_analyse = df.drop(columns=['Unnamed: 0.1', 'time', 'Unnamed: 0', 'floods', 'time_ok', 't'])

X = df_to_analyse.drop(columns=['floods_x4'])
y = df_to_analyse['floods_x4']

X_train = X.iloc[0:int(len(X) *.8)] 
X_test = X.iloc[int(len(X) *.8):int(len(X) *.9)]
X_val = X.iloc[int(len(X) *.9):]

y_train = y.iloc[0:int(len(y) *.8)]
y_test = y.iloc[int(len(y) *.8):int(len(y) *.9)]
y_val = y.iloc[int(len(y) *.9):]

In [4]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [15]:
# svc = SVC(
#     C= 10, ###
#     kernel= "sigmoid", ### Literal['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
#     degree=4, ###
#     gamma='auto', ### float | Literal['scale', 'auto'] = "scale",
#     coef0=0,
#     shrinking=True,
#     probability = False,
#     tol= 0.001,
#     cache_size= 200,
#     max_iter = 500,
#     # decision_function_shape ='ovr', ### Literal['ovo', 'ovr'] = "ovr", -> Ignored for Binary Classfication.
#     # break_ties= False, -> Ignored for Binary Classfication.
#     class_weight=weights,
#     verbose=1,
#     random_state=42,
# )

svc = LinearSVC(
    penalty='l1', ###: Literal['l1', 'l2'] = "l2",
    loss='squared_hinge', ### Literal['squared_hinge', 'hinge'] = "squared_hinge",
    dual='auto', ###: bool = True,
    tol= 0.0001,
    C= 10,
    # multi_class: Literal['ovr', 'crammer_singer'] = "ovr",
    fit_intercept = True,
    intercept_scaling= 1,
    class_weight=weights,
    verbose= 1,
    random_state=42,
    max_iter= 1000,
)

In [16]:
svc.fit(X_train, y_train)

from sklearn.metrics import balanced_accuracy_score

y_pred = svc.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

[LibLinear]....................................................................................................
optimization finished, #iter = 1000

Objective value = 91.093390
#nonzeros/#features = 132/4954
0.8458833619210977


/Users/tudor/opt/anaconda3/envs/phd_main/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
